<font size="+3"><strong>Analyzing Movie Data and Predicting Box Office Performance with the TMDb API</strong></font>

### On présente ici l'API TMDb utilisée pour extraire nos données. Nous décrivons comment nous avons collecté des données sur les films.

##### L'API TMDb (The Movie Database) est une API gratuite et ouverte permettant d'accéder aux données des films, des émissions de télévision et des personnes. Il donne accès à une mine d'informations sur les films, y compris les budgets, les revenus, les acteurs, l'équipe, les genres... En utilisant l'API TMDb, nous pouvons récupérer des données sur un grand nombre de films et les utiliser pour construire un modèle capable de prédire les recettes au box-office.

In [1]:
# Import libraries.
import time
import requests
import pandas as pd

## Importation

##### Pour pouvoir récupérer les informations d'un film, il faut connaitre l'identifiant de ce film dans la base. Le problème est que nous ne connaissons pas les identifiants présents dans la base. Nous avons donc d'abord essayé de récupérer les identifiants au hasard, ce qui a pris beaucoup trop de temps à charger. Il nous fallait donc une méthode alternative  nécessitant moins de requête.
##### Nous avons donc décidé de procéder page par page, en passant par le point d'accès discover. Discover renvoie diverses pages de films. Nous avons ensuite défini une plage de dates : nous nous sommes restreints aux films sortis entre 1900 et 2015 après avoir remarqué que plus un film est récent, moins on a d’informations sur ce dernier. Nous avons également trié les résultats par ordre décroissant de popularité, pour maximiser les chances de ne pas avoir de valeurs manquantes. Plus un film est populaire, plus on a d'informations sur celiu-ci.
##### Finalement, on parcourt toutes les pages à travers le point d'accès discover, et pour chacun des films qui apparaissent sur cette page, on effectue la requête pour obtenir les détails sur ce film.

In [2]:
def get_movies(api_key, base_url, params):
    """Request data from TMDb API.
    
    Parameters
    ----------
    api_key: string, 
        API key.
    
    base_url: string, 
        base URL for API request.
    
    params: dictionary, 
        list of tuples or bytes to send in the query string.
    
    Return
    ------
    movies: json, 
        lists of movies imported
    """
    
    # Initialize an empty list to store the movie data
    movies = []
    
    # Set a flag to indicate whether there are more pages to request
    more_pages = True
    
    # Start time and iteration index
    start_time = time.time()

    while more_pages:        
        # Make the API request
        response = requests.get(f'{base_url}discover/movie', params=params)

        # Check the status code to make sure the request was successful
        if response.status_code == 200:
            # Loop through the results and make a separate API request for each movie
            for movie in response.json()['results']:
                # Set the movie ID for the API request
                movie_id = movie['id']

                # Make the API request for the movie details
                movie_response = requests.get(f'{base_url}movie/{movie_id}', params={'api_key': api_key})

                # Check the status code to make sure the request was successful
                if movie_response.status_code == 200:
                    # Add the movie data to the list
                    movies.append(movie_response.json())

            # Update the page number for the next request
            params['page'] += 1

            # Check if there are more pages to request
            if params['page'] > response.json()['total_pages']:
                more_pages = False
        else:
            print(f'Request failed with status code {response.status_code}')
            break

        # Verbose printing
        if params['page'] % 25 == 0:
            print(f"Page {params['page']}. Load {len(movies)} movies. Elapsed time {time.time() - start_time} seconds.")

    return movies

In [3]:
# API key
api_key = "06af3e1b654bb9481c777bde394b620b"
# Set the base URL for the TMDB API
base_url = 'https://api.themoviedb.org/3/'
# Set the parameters for the API request
params = {
    'api_key': api_key,
    'language': 'en-US',
    'include_adult': 'false',
    'sort_by': 'popularity.desc',  # most popular movies
    'primary_release_date.gte': '1900-01-01',  # start date
    'primary_release_date.lte': '2015-12-31',  # end date
    'page': 1
}

movies = get_movies(api_key, base_url, params)

Page 25. Load 480 movies. Elapsed time 93.0046534538269 seconds.
Page 50. Load 980 movies. Elapsed time 189.97506618499756 seconds.
Page 75. Load 1480 movies. Elapsed time 286.50015687942505 seconds.
Page 100. Load 1980 movies. Elapsed time 381.6607096195221 seconds.
Page 125. Load 2480 movies. Elapsed time 476.96944546699524 seconds.
Page 150. Load 2980 movies. Elapsed time 573.2153375148773 seconds.
Page 175. Load 3480 movies. Elapsed time 668.552496433258 seconds.
Page 200. Load 3980 movies. Elapsed time 763.9174160957336 seconds.
Page 225. Load 4480 movies. Elapsed time 860.8392179012299 seconds.
Page 250. Load 4980 movies. Elapsed time 960.2945592403412 seconds.
Page 275. Load 5479 movies. Elapsed time 1057.0364940166473 seconds.
Page 300. Load 5979 movies. Elapsed time 1153.9656944274902 seconds.
Page 325. Load 6479 movies. Elapsed time 1251.696575641632 seconds.
Page 350. Load 6979 movies. Elapsed time 1349.213386297226 seconds.
Page 375. Load 7479 movies. Elapsed time 1448.0404

In [4]:
# Turn into DataFrame and save as pickle for further use.
# To avoid having to run again all the API requests.
movies_df = pd.DataFrame(movies)
movies_df.to_pickle("./data/movies_tmdb.pkl")

In [5]:
# Keep only non-zero budget and revenue rows.
movies_df = movies_df.query('budget > 0 and revenue > 0')
print(movies_df.shape)
movies_df.head(2)

(4974, 25)


,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/Yc9q6QuWrMp9nuDm5R8ExNqbEq.jpg,"{'id': 87096, 'name': 'Avatar Collection', 'po...",237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",https://www.avatar.com/movies/avatar,19995,tt0499549,en,Avatar,...,2009-12-15,2920357254,162,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Enter the world of Pandora.,Avatar,False,7.543,27045
3,False,/tuDhEdza074bA497bO9WFEPs6O6.jpg,"{'id': 420, 'name': 'The Chronicles of Narnia ...",180000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",,411,tt0363771,en,"The Chronicles of Narnia: The Lion, the Witch ...",...,2005-12-07,745013115,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Evil Has Reigned For 100 Years...,"The Chronicles of Narnia: The Lion, the Witch ...",False,7.102,8982
